In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy.io import fits
import astropy.units as u
import scipy.stats as ss

In [ ]:
import sys
sys.path.append("..")
import panco2 as p2

# C1

## C1 with Planck

In [ ]:
path = "./results/C1/Planck"
ppf = p2.PressureProfileFitter(
    f"{path}/input_map.fits",
    1, 5,
    0.05, 9e14,
    map_size=4.0 * 60,
    coords_center=SkyCoord("12h00m00s +00d00m00s")
)

pix_kpc, half_map_kpc = ppf.cluster.arcsec2kpc(ppf.pix_size), ppf.cluster.arcsec2kpc(ppf.map_size * 60 / 2)
beam_kpc = ppf.cluster.arcsec2kpc(600.0)
r_bins = np.concatenate(([pix_kpc], np.logspace(np.log10(beam_kpc), np.log10(1.1 * half_map_kpc), 4)))

ppf.define_model("binned", r_bins)

ppf.add_filtering(beam_fwhm=600.0)

P_bins = p2.utils.gNFW(r_bins, *ppf.cluster.A10_params)
ppf.define_priors(
    P_bins=[ss.loguniform(0.01 * P, 100.0 * P) for P in P_bins],
    conv=ss.norm(1.0, 0.05),
    zero=ss.norm(0.0, 1e-5)
)
ppf.dump_to_file(f"{path}/C1_planck.panco2")

plt.close("all")
fig, ax = plt.subplots()
ax.loglog(r_bins, P_bins, "o-")
for x in [pix_kpc, beam_kpc / 2, beam_kpc, half_map_kpc, half_map_kpc * np.sqrt(2)]:
    ax.axvline(x, 0, 1, color="k", ls=":")
ax.axvline(ppf.cluster.R_500, 0, 1, color="k", ls="--")

par_vec = np.append(P_bins, [1.0, 0.0])
_ = p2.results.plot_data_model_residuals(ppf, par_vec, smooth=0.5, cbar_fact=1e5, cbar_label="Compton $y \\times 10^5$")
_, ax = p2.results.plot_data_model_residuals_1d(ppf, par_vec=par_vec, y_fact=1e5, y_label="Compton $y \\times 10^5$")
ax.set_xscale("log")
ax.set_xlim(ppf.pix_size / 2, ppf.map_size * 60 / np.sqrt(2))

In [ ]:
_ = ppf.run_mcmc(
    30, 1e5, 4, n_check=1e3, max_delta_tau=0.02, min_autocorr_times=20, 
    out_chains_file=f"{path}/rawchains.npz", plot_convergence=f"{path}/mcmc_convergence.pdf"
)

In [ ]:
chains_clean = p2.results.load_chains(f"{path}/rawchains.npz", 500, 50, clip_percent=25.0, verbose=True)

In [ ]:
plt.close("all")
_ = p2.results.mcmc_trace_plot(chains_clean, filename=f"{path}/mcmc_trace.png")

In [ ]:
_ = p2.results.mcmc_corner_plot(chains_clean, ppf=ppf)

In [ ]:
meds = dict(chains_clean.median())
p2.results.plot_data_model_residuals(
    ppf, par_dic=meds, smooth=0.5, cbar_fact=1e5, lims="sym",
    cbar_label="Compton $y \\times 10^5$", filename=f"{path}/data_model_residuals_maps.pdf"
)
p2.results.plot_data_model_residuals_1d(
    ppf, par_dic=meds, y_fact=1e5, plot_beam=False, 
    y_label="Compton $y \\times 10^5$", filename=f"{path}/data_model_residuals_profiles.pdf", x_log=True
)

In [ ]:
r_range = np.logspace(np.log10(pix_kpc), np.log10(half_map_kpc * np.sqrt(2)), 100)
fig, ax = p2.results.plot_profile(chains_clean, ppf, r_range=r_range, label="panco2")
ax.plot(r_range, p2.utils.gNFW(r_range, *ppf.cluster.A10_params), "k--", label="Truth")
ax.legend(frameon=False)
fig.savefig(f"{path}/pressure_profile.pdf")

## C1 with SPT

In [ ]:
path = "./results/C1/SPT/"
ppf = p2.PressureProfileFitter(
    f"{path}/input_map.fits",
    1, 5,
    0.05, 9e14,
    map_size=1.0 * 60,
    coords_center=SkyCoord("12h00m00s +00d00m00s")
)
ppf.sz_rms *= 2

pix_kpc, half_map_kpc = ppf.cluster.arcsec2kpc(ppf.pix_size), ppf.cluster.arcsec2kpc(ppf.map_size * 60 / 2)
beam_kpc = ppf.cluster.arcsec2kpc(75.0)
r_bins = np.concatenate(([pix_kpc], np.logspace(np.log10(beam_kpc), np.log10(1.1 * half_map_kpc), 4)))
#r_bins = np.logspace(np.log10(pix_kpc), np.log10(half_map_kpc), 6)

ppf.define_model("binned", r_bins)

ppf.add_filtering(beam_fwhm=75.0)

P_bins = p2.utils.gNFW(r_bins, *ppf.cluster.A10_params)
ppf.define_priors(
    P_bins=[ss.loguniform(0.01 * P, 100.0 * P) for P in P_bins],
    conv=ss.norm(1.0, 0.05),
    zero=ss.norm(0.0, 1e-5)
)
ppf.dump_to_file(f"{path}/C1_spt.panco2")

plt.close("all")
fig, ax = plt.subplots()
ax.loglog(r_bins, P_bins, "o-")
for x in [pix_kpc, beam_kpc / 2, beam_kpc, half_map_kpc, half_map_kpc * np.sqrt(2)]:
    ax.axvline(x, 0, 1, color="k", ls=":")
ax.axvline(ppf.cluster.R_500, 0, 1, color="k", ls="--")

par_vec = np.append(P_bins, [1.0, 0.0])
_ = p2.results.plot_data_model_residuals(ppf, par_vec, smooth=1.0, cbar_fact=1e5, cbar_label="Compton $y \\times 10^5$")
_, ax = p2.results.plot_data_model_residuals_1d(ppf, par_vec=par_vec, y_fact=1e5, y_label="Compton $y \\times 10^5$", x_log=True)

In [ ]:
_ = ppf.run_mcmc(
    30, 1e5, 4, n_check=1e3, max_delta_tau=0.02, min_autocorr_times=20, 
    out_chains_file=f"{path}/rawchains.npz", plot_convergence=f"{path}/mcmc_convergence.pdf"
)

In [ ]:
chains_clean = p2.results.load_chains(f"{path}/rawchains.npz", 500, 20, clip_percent=25.0, verbose=True)

In [ ]:
plt.close("all")
_ = p2.results.mcmc_trace_plot(chains_clean, filename=f"{path}/mcmc_trace.png")

In [ ]:
_ = p2.results.mcmc_corner_plot(chains_clean, ppf=ppf)

In [ ]:
meds = dict(chains_clean.median())
p2.results.plot_data_model_residuals(
    ppf, par_dic=meds, smooth=1.0, cbar_fact=1e5, lims="sym",
    cbar_label="Compton $y \\times 10^5$", filename=f"{path}/data_model_residuals_maps.pdf"
)
p2.results.plot_data_model_residuals_1d(
    ppf, par_dic=meds, y_fact=1e5, plot_beam=False, 
    y_label="Compton $y \\times 10^5$", filename=f"{path}/data_model_residuals_profiles.pdf", x_log=True
)

In [ ]:
r_range = np.logspace(np.log10(pix_kpc), np.log10(half_map_kpc * np.sqrt(2)), 100)
fig, ax = p2.results.plot_profile(chains_clean, ppf, r_range=r_range, label="panco2")
ax.plot(r_range, p2.utils.gNFW(r_range, *ppf.cluster.A10_params), "k--", label="Truth")
ax.legend(frameon=False)
fig.savefig(f"{path}/pressure_profile.pdf")

# C2

## C2 with NIKA2

In [ ]:
path = "./results/C2/NIKA2/"
make_sim_map_nika2(0.5, 5e14, f"{path}/map_C2_nk2.fits", conv=-12.0, zero=0.0, map_size=8.5)

In [ ]:
mapview.fitsview(f"{path}/map_C2_nk2.fits", 1, smooth=1.0, fwhm=3.0, imrange="sym")

In [ ]:
ppf = p2.PressureProfileFitter(
    f"{path}/map_C2_nk2.fits",
    1,
    5,
    0.5,
    M_500=5e14,
    map_size=6.5
)

pix_kpc, map_kpc = ppf.cluster.arcsec2kpc(ppf.pix_size), ppf.cluster.arcsec2kpc(60 * ppf.map_size)
r_bins = [pix_kpc]
while r_bins[-1] < map_kpc / 2:
    r_bins.append(3 * r_bins[-1])
#_ =  r_bins.pop(1)
r_bins = np.array(r_bins)

ppf.define_model("binned", r_bins)

P_bins = p2.utils.gNFW(ppf.model.r_bins, *ppf.cluster.A10_params)
ppf.define_priors(
    P_bins=[ss.loguniform(0.01 * P, 100.0 * P) for P in P_bins],
    conv=ss.norm(-12.0, 1.2),
    zero=ss.norm(0.0, 1e-3)
)

tf = Table.read("./example_data/NIKA2/nk2_tf.fits")
ppf.add_filtering(
    beam_fwhm=18.0,
    k=tf["k"].to("arcsec-1").value,
    tf=tf["tf_2mm"].value,
    pad=60,
)
ppf.dump_to_file(f"{path}/C2_nk2.panco2")

In [ ]:
fig, ax = plt.subplots()
ax.semilogx(r_bins, np.zeros_like(r_bins), "o-")
for x in [ppf.cluster.arcsec2kpc(ppf.pix_size), ppf.cluster.arcsec2kpc(ppf.beam_fwhm / 2), ppf.cluster.arcsec2kpc(ppf.map_size * 30)]:
    ax.axvline(x, 0, 1, color="k", ls="--")

par_vec = np.append(P_bins, [-12.0, 0.0])
_ = p2.results.plot_data_model_residuals(ppf, par_vec, smooth=1.0, cbar_fact=1e3)

In [ ]:
_ = ppf.run_mcmc(
    30, 5e4, 4, n_check=1e3, max_delta_tau=0.02, min_autocorr_times=50, 
    out_chains_file=f"{path}/rawchains_C2_nk2.npz", plot_convergence=f"{path}/mcmc_convergence.pdf"
)

In [ ]:
chains_clean = p2.results.load_chains(f"{path}/rawchains_C2_nk2.npz", 500, 25, clip_percent=25.0, verbose=True)

In [ ]:
_ = p2.results.mcmc_trace_plot(chains_clean, filename=f"{path}/mcmc_trace.pdf")
_ = p2.results.mcmc_corner_plot(chains_clean, ppf=ppf, filename=f"{path}/mcmc_corner.pdf")
_ = p2.results.mcmc_matrices_plot(chains_clean, ppf, filename=f"{path}/mcmc_corrmat.pdf")

In [ ]:
meds = dict(chains_clean.median())
p2.results.plot_data_model_residuals(
    ppf, par_dic=meds, smooth=1.0, cbar_fact=1e3, lims="sym",
    cbar_label="NIKA2 150 GHz [mJy/beam]", filename=f"{path}/data_model_residuals_maps.pdf"
)
p2.results.plot_data_model_residuals_1d(
    ppf, par_dic=meds, y_fact=1e3, plot_beam=False, 
    y_label="NIKA2 150 GHz [mJy/beam]", filename=f"{path}/data_model_residuals_profiles.pdf"
)

In [ ]:
r_range = np.logspace(np.log10(pix_kpc), np.log10(map_kpc / np.sqrt(2)), 100)
fig, ax = p2.results.plot_profile(chains_clean, ppf, r_range=r_range, label="panco2")
ax.legend(frameon=False)
fig.savefig(f"{path}/pressure_profile.pdf")

## C2 with SPT

In [ ]:
path = "./results/C2/SPT/"
make_sim_map_spt(0.5, 5e14, f"{path}/map_C2_spt.fits", conv=1.0, zero=0.0, map_size=90.0)

In [ ]:
mapview.fitsview(f"{path}/map_C2_spt.fits", 1, fwhm=5.0, crop=30*u.arcmin, smooth=0.5)

In [ ]:
ppf = p2.PressureProfileFitter(
    f"{path}/map_C2_spt.fits",
    1,
    5,
    0.5,
    M_500=5e14,
    map_size=30.0
)

pix_kpc, map_kpc = ppf.cluster.arcsec2kpc(ppf.pix_size), ppf.cluster.arcsec2kpc(60 * ppf.map_size)
r_bins = [pix_kpc]
while r_bins[-1] < map_kpc / 2:
    r_bins.append(4 * r_bins[-1])
#_ =  r_bins.pop(1)
r_bins = np.array(r_bins)

ppf.define_model("binned", r_bins)

P_bins = p2.utils.gNFW(ppf.model.r_bins, *ppf.cluster.A10_params)
ppf.define_priors(
    P_bins=[ss.loguniform(0.01 * P, 100.0 * P) for P in P_bins],
    conv=ss.norm(1.0, 0.05),
    zero=ss.norm(0.0, 1e-5)
)

ppf.add_filtering(
    beam_fwhm=75.0,
    pad=0,
)
ppf.dump_to_file(f"{path}/C2_spt.panco2")

In [ ]:
fig, ax = plt.subplots()
ax.semilogx(r_bins, np.zeros_like(r_bins), "o-")
for x in [ppf.cluster.arcsec2kpc(ppf.pix_size), ppf.cluster.arcsec2kpc(ppf.beam_fwhm / 2), ppf.cluster.arcsec2kpc(ppf.map_size * 30)]:
    ax.axvline(x, 0, 1, color="k", ls="--")

par_vec = np.append(P_bins, [1.0, 0.0])
_ = p2.results.plot_data_model_residuals(ppf, par_vec, smooth=0.5, cbar_fact=1e5, cbar_label="Compton $y \\times 10^5$")

In [ ]:
_ = ppf.run_mcmc(
    30, 5e4, 4, n_check=1e3, max_delta_tau=0.02, min_autocorr_times=50, 
    out_chains_file=f"{path}/rawchains_C2_spt.npz", plot_convergence=f"{path}/mcmc_convergence.pdf"
)

In [ ]:
chains_clean = p2.results.load_chains(f"{path}/rawchains_C2_spt.npz", 500, 25, clip_percent=25.0, verbose=True)

In [ ]:
_ = p2.results.mcmc_trace_plot(chains_clean, filename=f"{path}/mcmc_trace.pdf")
_ = p2.results.mcmc_corner_plot(chains_clean, ppf=ppf, filename=f"{path}/mcmc_corner.pdf")
_ = p2.results.mcmc_matrices_plot(chains_clean, ppf, filename=f"{path}/mcmc_corrmat.pdf")

In [ ]:
meds = dict(chains_clean.median())
p2.results.plot_data_model_residuals(
    ppf, par_dic=meds, smooth=1.0, cbar_fact=1e5, lims="sym",
    cbar_label="Compton $y \\times 10^5$", filename=f"{path}/data_model_residuals_maps.pdf"
)
p2.results.plot_data_model_residuals_1d(
    ppf, par_dic=meds, y_fact=1e5, plot_beam=False, 
    y_label="Compton $y \\times 10^5$", filename=f"{path}/data_model_residuals_profiles.pdf"
)

In [ ]:
r_range = np.logspace(np.log10(pix_kpc), np.log10(map_kpc / np.sqrt(2)), 100)
fig, ax = p2.results.plot_profile(chains_clean, ppf, r_range=r_range, label="panco2")
ax.legend(frameon=False)
fig.savefig(f"{path}/pressure_profile.pdf")